In [1]:
import sys
sys.path.append('..')


from torch.utils.data import DataLoader
from dataset import AEDataset
from lightning_model import LitAE

In [2]:
from types import SimpleNamespace
config = {
    'enc_filters': (64, 128, 128, 256),
    'latent_dim': 128,
    'enc_bn': True,
    'dec_features': (256, 256),
    'n_pts': 256,
    'dec_bn': False,
}
config = SimpleNamespace(**config)

In [3]:
checkpoint = 'lightning_logs/version_5/checkpoints/epoch=10-step=12276-saved.ckpt'
autoencoder = LitAE.load_from_checkpoint(checkpoint, config=config)

In [17]:
autoencoder.autoencoder.encoder

EncoderPointNet(
  (model): Sequential(
    (0): Conv1d(39, 64, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01, inplace=True)
    (3): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01, inplace=True)
    (6): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
    (7): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01, inplace=True)
    (9): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
    (10): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.01, inplace=True)
    (12): Conv1d(256, 128, kernel_size=(1,), stride=(1,))
    (13): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14):

In [4]:
autoencoder.eval()

LitAE(
  (autoencoder): PointAE(
    (encoder): EncoderPointNet(
      (model): Sequential(
        (0): Conv1d(39, 64, kernel_size=(1,), stride=(1,))
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.01, inplace=True)
        (3): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
        (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): LeakyReLU(negative_slope=0.01, inplace=True)
        (6): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
        (7): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (8): LeakyReLU(negative_slope=0.01, inplace=True)
        (9): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
        (10): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (11): LeakyReLU(negative_slope=0.01, inplace=True)
        (12): Conv1d(256, 128, kernel_size=(1,), stride=(1

In [5]:
root = '..'
dataset_name = 'shapenetcorev2'

test_dataset = AEDataset(root=root, dataset_name=dataset_name, num_points=config.n_pts, split='test')
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4)

In [6]:
next(iter(test_loader))['points_encoded'].shape

torch.Size([1, 39, 256])

In [7]:
from visualize import mitsuba

In [8]:
def create_comparsion(index):
    ori_pts = test_dataset[index]['points'].numpy().T
    out = autoencoder.autoencoder(test_dataset[index]['points_encoded'].unsqueeze(0))
    recon_pts = out[0].detach().cpu().numpy().T
    
    mitsuba(ori_pts, f'results/original{index}.xml')
    mitsuba(recon_pts, f'results/recon{index}.xml')

In [9]:
out = autoencoder.autoencoder(next(iter(test_loader))['points_encoded'])

In [13]:
autoencoder.autoencoder.encoder(next(iter(test_loader))['points_encoded']).shape

torch.Size([1, 128])